In [2]:
from copy import deepcopy
from typing import List, Tuple

with open('connections.txt', 'r') as f:
    lines = f.readlines()[2:]

class Noeud:

    def __init__(self, origine, destination, temps_trajet):
        self.origine = origine
        self.destination = destination
        self.temps_trajet = temps_trajet
        self.lettre = None

    def __str__(self):
        return f"{self.origine} -> {self.destination} [travel-time={self.temps_trajet} hrs]"
    
    def __repr__(self):
        return self.__str__()
    
    @classmethod
    def from_string(cls, line: str):
        parts = line.strip().split()
        return cls(parts[1], parts[3], float(parts[5]))

def lister_les_connections():
    for line in lines:
        Connections.parse_connection(line)


class Connections:
    registre = {}

    @classmethod
    def parse_connection(cls, line: str):
        connection = Noeud.from_string(line)
        cls.enregister(connection)

    @classmethod
    def enregister(cls, connection):
        cls.registre.setdefault(connection.origine, []).append(connection)
    
    @classmethod
    def prochaine_destination(cls, origine: str) -> List[Noeud]:
        return cls.registre.get(origine, [])
    
    

class Chemin:
    compteur = 0
    registre = {}

    def __init__(self):
        self.pk = self.compteur
        self.compteur += 1
        self.origine = "S"
        self.destination = "E"
        self.etapes = []
        self._temps_total = 0.0
        self.lettres = []
        self.actif = True
        self.prochaine_etape = None

    @classmethod
    def obtenir_les_chemins_actifs(cls) -> List['Chemin']:
        return [chemin for chemin in cls.registre.values() if chemin.actif]

    @classmethod
    def obtenir_les_chemins_definitifs(cls) -> List['Chemin']:
        return [chemin for chemin in cls.registre.values() if not chemin.actif]
    
    @classmethod
    def ajouter_chemin(cls):
        chemin = cls()
        cls.registre[chemin.pk] = chemin
        return chemin
    
    @classmethod
    def dupliquer_chemin(cls, pk: int):
        obj = cls.registre.get(pk)
        if not obj:
            raise ValueError(f"Chemin avec pk {pk} n'existe pas.")
        
        new_ = cls.ajouter_chemin()
        new_.etapes = deepcopy(obj.etapes)
        new_._temps_total = obj._temps_total
        new_.lettres = deepcopy(obj.lettres)
        return new_
        
    def ajouter_etape(self, connection: Noeud):
        if self.actif:
            self.etapes.append(connection.destination)
            self._temps_total += connection.temps_trajet
            if connection.lettre:
                self.lettres.append(connection.lettre)
            self.prochaine_etape = connection.destination
                
            # Désactiver si la destination finale est atteinte
            if connection.destination == self.destination:
                self.actif = False
    
    @staticmethod
    def ajouter_noeud(chemin: 'Chemin'):
        destinations_suivantes = Connections.prochaine_destination(chemin.etapes[-1])
        for cnt, destination in enumerate(destinations_suivantes):
            if cnt % len(destinations_suivantes) == 0:
                chemin.ajouter_etape(destination)
            else:  
                new_chemin = Chemin.dupliquer_chemin(chemin.pk)
                new_chemin.ajouter_etape(destination)


class Planificateur:
    Chemin.ajouter_chemin()
        
    @classmethod
    def construire_tous_les_chemins(cls):
        chemins_actifs = Chemin.obtenir_les_chemins_actifs()
        while chemins_actifs:
            # nouveaux_chemins = []
            for chemin in chemins_actifs:
                Chemin.ajouter_noeud(chemin)
        return Chemin.obtenir_les_chemins_definitifs()



    

    

In [3]:
lister_les_connections()

plannificateur = Planificateur()
plannificateur.construire_tous_les_chemins()

IndexError: list index out of range

{'S': [S -> A [travel-time=4.0 hrs],
  S -> B [travel-time=5.0 hrs],
  S -> C [travel-time=8.0 hrs],
  S -> H [travel-time=6.0 hrs]],
 'A': [A -> B [travel-time=2.0 hrs],
  A -> D [travel-time=7.0 hrs],
  A -> H [travel-time=7.0 hrs]],
 'B': [B -> C [travel-time=3.0 hrs],
  B -> D [travel-time=4.0 hrs],
  B -> E [travel-time=14.0 hrs],
  B -> H [travel-time=8.0 hrs],
  B -> I [travel-time=6.0 hrs]],
 'C': [C -> D [travel-time=6.0 hrs], C -> K [travel-time=8.0 hrs]],
 'D': [D -> F [travel-time=2.0 hrs],
  D -> E [travel-time=8.0 hrs],
  D -> I [travel-time=5.0 hrs]],
 'F': [F -> G [travel-time=3.0 hrs],
  F -> E [travel-time=6.0 hrs],
  F -> J [travel-time=5.0 hrs]],
 'G': [G -> E [travel-time=2.0 hrs], G -> J [travel-time=6.0 hrs]],
 'J': [J -> E [travel-time=4.0 hrs]],
 'H': [H -> D [travel-time=7.0 hrs],
  H -> B [travel-time=8.0 hrs],
  H -> A [travel-time=7.0 hrs]],
 'I': [I -> F [travel-time=3.0 hrs], I -> D [travel-time=5.0 hrs]],
 'K': [K -> E [travel-time=9.0 hrs]]}